In [1]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, make_scorer
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, StratifiedKFold
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline

In [2]:
# LEER LOS DATOS 
file_in = pd.read_csv('dead_due_to_AHE.csv') #lee archivo 
file_in = file_in.drop(['subject_id', 'hadm_id', 'icustay_id'], axis= 'columns') #elimino las columnas que no ocupo 

# IMPUTACIÓN DE DATOS 
x = file_in.drop(['dead'], axis='columns') #para mis variables de entrada soalamente elimino la varaible a predecir 
y= file_in.dead #me quedo con la variable a predecir en y 
columnas = x.columns 

# DIVIDIR CONJUNTO DE ENTRENAMIENTO, VALIDACIÓN Y PRUEBA 
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, )
print('X_train:', x_train.shape, 'X_test:', x_test.shape) 


#data = x.replace(" ", np.NaN)
#values = data.values
imp= SimpleImputer(missing_values=np.nan, strategy='mean').fit(x_train)
#imp = IterativeImputer(max_iter=10, random_state=0)
#imp = KNNImputer(n_neighbors=5, weights= 'uniform')
#imp = MissForest()
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)
print('X_train:', x_train.shape, 'X_test:', x_test.shape)

X_train: (3563, 40) X_test: (1528, 40)
X_train: (3563, 40) X_test: (1528, 40)


In [3]:
print(type(x_train))
x_train_imputado = pd.DataFrame(x_train, columns=[columnas]) #convirtiendo el arreglo en dataframe 
x_test_imputado = pd.DataFrame(x_test, columns=[columnas])
x_train_imputado

<class 'numpy.ndarray'>


,age,kg,creatinine,sexo,first_sicu,first_micu,first_ccu,first_tsicu,first_csru,first_nicu,...,resprate_max,tempc_min,tempc_mean,tempc_max,spo2_min,spo2_mean,spo2_max,pao2_min,pao2_mean,pao2_max
0,48.0,104.099998,1.1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,36.0,36.722221,37.296296,37.833332,94.0,98.622222,100.0,87.000000,166.833333,345.000000
1,69.0,89.800003,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,23.0,36.299999,36.528353,36.900002,93.0,97.777778,100.0,124.000000,247.333333,368.000000
2,75.0,83.400002,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,28.0,34.055557,36.225343,38.666666,76.0,97.939655,100.0,55.000000,99.111111,152.000000
3,87.0,87.800003,1.8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,27.0,35.166668,36.305556,37.111113,93.0,97.692308,100.0,84.332453,154.179360,303.195955
4,19.0,66.800003,0.8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,24.0,35.555556,36.439815,36.833335,97.0,99.163934,100.0,84.332453,154.179360,303.195955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558,60.0,48.000000,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,43.0,36.277779,36.937198,38.000001,85.0,98.347368,100.0,84.332453,154.179360,303.195955
3559,85.0,80.699997,0.6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,36.0,36.222221,37.221839,38.200001,92.0,96.688525,100.0,69.000000,228.000000,412.000000
3560,66.0,99.000000,0.8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,54.0,35.166668,37.307808,38.166665,92.0,98.828179,100.0,76.000000,148.315789,344.000000
3561,50.0,79.400002,0.6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,34.0,34.799999,37.478118,39.799999,90.0,97.829365,100.0,66.000000,171.333333,430.000000


In [4]:
# ESTANDARIZACIÓN DE DATOS 
estandarizar = ['age', 'kg', 'creatinine','heartrate_min', 'heartrate_mean', 'heartrate_max', 'sysbp_min',
       'sysbp_mean', 'sysbp_max', 'diasbp_min', 'diasbp_mean', 'diasbp_max',
       'meanbp_min', 'meanbp_mean', 'meanbp_max', 'resprate_min','resprate_mean', 'resprate_max', 'tempc_min', 'tempc_mean', 'tempc_max',
       'spo2_min', 'spo2_mean', 'spo2_max', 'pao2_min', 'pao2_mean','pao2_max']
dummies = ['sexo', 'first_sicu', 'first_micu',
       'first_ccu', 'first_tsicu', 'first_csru', 'first_nicu', 'last_sicu','last_micu', 'last_ccu', 'last_tsicu', 'last_csru', 'last_nicu']
scaler = StandardScaler().fit(x_train_imputado)
x_train_escalado = scaler.fit_transform(x_train_imputado)
x_test_escalado = scaler.fit_transform(x_test_imputado)

print('X_train:', x_train_escalado.shape, 'X_test:', x_test_escalado.shape)

X_train: (3563, 40) X_test: (1528, 40)


In [5]:
# DEFINIR MODELO A UTILIZAR 
#RF
modeloXGB = xgb.XGBClassifier(use_label_encoder=False)
modeloRF = RandomForestClassifier()

#DEFINIR PARÁMETROS A ITERAR

parametrosRF = {'n_estimators': range(10, 200, 10), 'criterion':['gini', 'entropy'], 'max_depth': range(1,10,1), 
              'max_features':['auto', 'sqrt', 'log2'],  'min_impurity_decrease':[1e-4, 1e-7], 
              'class_weight':['balanced', 'balanced_subsample']} # PARA RF

parametrosRFprueba = {'n_estimators': range(10, 200, 10),'max_depth': range(1,10,1),'class_weight':['balanced']}

ratio = float(np.sum(y_train == 0)) / np.sum(y_train==1)
                      
parametrosXGB = {'loss':['deviance', 'exponential'], 'criterion':['friedman_mse', 'mse', 'mae'], 'max_features':['auto', 'sqrt',
              'log2'], 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05],"colsample_bytree" : [0.4, 0.8],
                    "subsample" : [0.8, 0.4], "reg_alpha" : [0.5], "reg_lambda": [2],
                    "objective": ['binary:logistic'],  'max_depth': range(1,10,1), "gamma":[10],"rate_drop": [0.5, 0.3],
                    "seed": [422], "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosXGBprueba = { 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05], 'max_depth': range(1,10,1),
                        "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

#ENTRENAR GRID SEARCH 
#TIPOS DE SCORE 
balanced = class_weight.compute_sample_weight('balanced', y_train)
RKF = RepeatedKFold(n_splits=5, n_repeats=2, random_state=422) #probar los CROSS VALIDATION
SKF = StratifiedKFold(n_splits=5, shuffle= True, random_state=422)
grid = GridSearchCV(estimator= modeloRF, param_grid= parametrosRFprueba, cv=RKF, scoring='roc_auc', verbose=2, refit= True)

In [6]:
grid.fit(x_train, y_train, sample_weight = balanced)

Fitting 10 folds for each of 171 candidates, totalling 1710 fits
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1,

[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=110; total time=   0.0s
[CV] END class_weight=balanced, max_depth=1, n_estimators=120; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=120; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=120; total time=   0.1s
[CV] END class_weight=balanced, max_depth=1, n_estimators=120; total time=   0.1s
[CV] END class_w

[CV] END class_weight=balanced, max_depth=2, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanced, max_depth=2, n_estimators=30; total time=   0.0s
[CV] END class_weight=balanc

[CV] END class_weight=balanced, max_depth=2, n_estimators=120; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=120; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=120; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.2s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.2s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_weight=balanced, max_depth=2, n_estimators=130; total time=   0.1s
[CV] END class_w

[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=40; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=50; total time=   0.0s
[CV] END class_weight=balanced, max_depth=3, n_estimators=50; total time=   0.0s
[CV] END class_weight=balanc

...

[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.7s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.7s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=7, n_estimators=190; total time=   0.6s
[CV] END class_weight=balanced, max_depth=8, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=8, n_estimators=10; total time=   0.0s
[CV] END class_wei

[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=100; total time=   0.4s
[CV] END class_weight=balanced, max_depth=8, n_estimators=110; total time=   0.4s
[CV] END class_weight=balanced, max_depth=8, n_estimators=110; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=110; total time=   0.3s
[CV] END class_weight=balanced, max_depth=8, n_estimators=110; total time=   0.3s
[CV] END class_w

[CV] END class_weight=balanced, max_depth=9, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=10; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanced, max_depth=9, n_estimators=20; total time=   0.0s
[CV] END class_weight=balanc

[CV] END class_weight=balanced, max_depth=9, n_estimators=110; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=110; total time=   0.3s
[CV] END class_weight=balanced, max_depth=9, n_estimators=110; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=110; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=110; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.5s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.4s
[CV] END class_weight=balanced, max_depth=9, n_estimators=120; total time=   0.4s
[CV] END class_w

GridSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=5, random_state=422),
             estimator=RandomForestClassifier(),
             param_grid={'class_weight': ['balanced'],
                         'max_depth': range(1, 10),
                         'n_estimators': range(10, 200, 10)},
             scoring='roc_auc', verbose=2)

In [7]:
# METRICAS DEL MODELO 
auc_train = grid.best_score_
std_train = grid.cv_results_['std_test_score'][grid.best_index_]
best_params = grid.best_params_
best_estimator = grid.best_estimator_

print("auc:",auc_train)
print("std_validation:",std_train)
print('best estimator:', best_estimator)
print('best parameters:',best_params)

auc: 0.8346008440256238
std_validation: 0.014479541302880587
best estimator: RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=150)
best parameters: {'class_weight': 'balanced', 'max_depth': 9, 'n_estimators': 150}
